In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import time # TIME PROCESSING
import os # LOADING & EXPORTING DATA
import datetime
from kaggle_secrets import UserSecretsClient

In [2]:
!pip install nba_api
from nba_api.stats.static import players, teams
from nba_api.stats.endpoints import teamgamelog, playergamelog

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0.0/284.9 kB ? eta -:--:--

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.9/284.9 kB 9.5 MB/s eta 0:00:00


In [3]:
if os.path.exists('/root/.kaggle/'):
    pass
else:
    os.mkdir('/root/.kaggle/')

user_secrets = UserSecretsClient()
secret_value_key = user_secrets.get_secret("kaggle_key")
secret_value_user = user_secrets.get_secret("kaggle_username")

with open('/root/.kaggle/kaggle.json', 'w') as fid:
    fid.writelines(f'{{"username":"{secret_value_user}","key":"{secret_value_key}"}}')

!chmod 600 /root/.kaggle/kaggle.json

In [4]:
dict_players = players.get_players()
df_players = pd.DataFrame(dict_players)
df_players.to_csv('/kaggle/working/NBA_PLAYERS.csv', index=False)

In [5]:
dict_teams = teams.get_teams()
df_teams = pd.DataFrame(dict_teams)
df_teams.to_csv('/kaggle/working/NBA_TEAMS.csv', index=False)

In [6]:
str_today_date = datetime.date.today().strftime('%Y-%m-%d')

In [7]:
list_nba_team_ids = df_teams['id'].drop_duplicates().tolist()

list_df_games = []

for team_id in list_nba_team_ids:
    time.sleep(20)
    xx = teamgamelog.TeamGameLog(team_id=team_id)
    df_team_games = xx.get_data_frames()[0]
    list_df_games.append(df_team_games)


df_games = pd.DataFrame(columns = list_df_games[0].columns)

for df_game in list_df_games:
    df_games = pd.concat([df_games,df_game])

df_games['DATE_ADDED'] = str_today_date
df_games.to_csv('/kaggle/working/NBA_GAMES.csv', index=False)

<ipython-input-7-44106445e0ed>:15: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_games = pd.concat([df_games,df_game])


In [8]:
list_nba_player_ids = pd.DataFrame(players.get_active_players())['id'].drop_duplicates().tolist()

list_df_player_games = []

for player_id in list_nba_player_ids:
    time.sleep(20)
    xx = playergamelog.PlayerGameLog(player_id=player_id)
    df_player_games = xx.get_data_frames()[0]
    list_df_player_games.append(df_player_games)

df_player_games = pd.DataFrame(columns = list_df_player_games[0].columns)

for df_player_game in list_df_player_games:
    df_player_games = pd.concat([df_player_games,df_player_game])
df_player_games['DATE_ADDED'] = str_today_date
df_player_games.to_csv('/kaggle/working/NBA_PLAYER_GAMES.csv', index=False)

<ipython-input-8-bd7d4f617cba>:14: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_player_games = pd.concat([df_player_games,df_player_game])


In [9]:
with open('/kaggle/working/dataset-metadata.json', 'w') as json_fid:
    json_fid.write('{\n  "title": "Update NBA_DATA",\n  "id": "salikhussaini49/nba-data",\n  "licenses": [{"name": "CC0-1.0"}]}')

In [ ]:
!kaggle datasets version -m new